# Quasi Majoranas in 3D


In [ ]:
import hpc05

client, dview, lview = hpc05.start_remote_and_connect(200, profile='pbs',
                                               folder='~/Work/induced_gap_B_field/',
                                               timeout=300, del_old_ipcluster=True)

# client, dview, lview = hpc05.connect.connect_ipcluster(50, profile='pbs2',
#                                                       folder='~/Work/induced_gap_B_field/',
#                                                       timeout=300)


In [ ]:
# 1. Standard library imports
import asyncio
from copy import copy
from functools import partial
from operator import itemgetter

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np

# 3. Internal imports
import funcs
import common
import adaptive_tools

# hv.notebook_extension('matplotlib')
print(kwant.__version__)

import adaptive
adaptive.notebook_extension()

# Wire with intrinsic SC

In [ ]:
syst_pars = dict(a=10, L=3000, r=35, shape='circle', with_leads=True, L_barrier=0)

params = dict(alpha=70, B_x=0.5, B_y=0, B_z=0, Delta=0.25, g=50, 
              orbital=True, mu_sc=100, mu=10, mu_lead=10, c_tunnel=3/4,
              V=lambda *_: 0,
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0, **funcs.constants.__dict__)

syst = funcs.make_simple_3d_wire(**syst_pars)

kwant.plot(syst);

# Wire with external SC

In [ ]:
syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=0,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

params = dict(alpha=70, B_x=0.5, B_y=0, B_z=0, Delta=60, g=50,
              orbital=True, mu_sc=100, mu=10, mu_lead=10, c_tunnel=3/4,
              V=lambda *_: 0,
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0, **funcs.constants.__dict__)


syst = funcs.make_3d_wire(**syst_pars)

kwant.plot(syst);

# Topological phase diagram

In [ ]:
from functools import partial

lead_pars = dict(a=10, angle=0, coverage_angle=135, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True,
                 rotate_spin_orbit=False)

params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=10, mu_lead=10, mu_sc=100, c_tunnel=3/4, V=lambda *x: 0,
              intrinsic_sc=True,
              **funcs.constants.__dict__)



def fix_params(params, lead_pars):
    params = copy(params)
    lead_pars = copy(lead_pars)
    params['Delta'] = 0.25
    lead_pars['r'] = lead_pars['r1']
    for k in ['coverage_angle', 'angle', 'r1', 'r2', 'A_correction', 'with_shell']:
        lead_pars.pop(k, None)
    lead_pars['superconducting'] = True
    return params, lead_pars


def get_pfaffians(x, keys, val, params=params, lead_pars=lead_pars):
    import common
    import funcs
    import collections
    import scipy.sparse.linalg as sla

    params = common.parse_params({**params, **val})

    if not isinstance(x, collections.Iterable):
        x = [x]
    for k, v in zip(keys, x):
        params[k] = v

    if params['intrinsic_sc']:
        make_lead = funcs.make_simple_lead
        params, lead_pars = fix_params(params, lead_pars)
    else:
        make_lead = funcs.make_lead

    lead = make_lead(**lead_pars).finalized()

    return funcs.calculate_pfaffian(lead, params)


vals = common.named_product(orbital=[True, False],
                            intrinsic_sc=[True, False])

learners = []
for val in vals:
    learner = adaptive_tools.Learner2D(partial(get_pfaffians, val=val, keys=['B_x', 'mu_lead']), ([0, 4], [7, 40]))
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)

In [ ]:
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)
runner.live_info()

In [ ]:
[l.loss() for l in learners]

In [ ]:
learner._loss

In [ ]:
runner.task.print_stack()

In [ ]:
runner.task.print_stack(), [l.npoints for l in learners]

In [ ]:
%%output size=120
hv.HoloMap(
    {tuple(val.values()): l.plot() * hv.HLine(14) for l, val in zip(learners, vals)},
    kdims=list(vals[0].keys())).layout().cols(2)

In [ ]:
learner.loss()

# Wavefunctions in the lead

In [ ]:
lead = syst.leads[0]
funcs.plot_wfs_in_cross_section(lead, params, 0)

# Finite sized system spectrum

In [ ]:
%%px --local
def fix_simple_params(params, syst_pars):
    from copy import copy
    params = copy(params)
    syst_pars = copy(syst_pars)
    params['Delta'] = 0.25
    syst_pars['r'] = syst_pars['r1']
    for k in ['coverage_angle', 'angle', 'r1', 'r2',
              'A_correction', 'with_shell', 'onsite_disorder']:
        syst_pars.pop(k, None)
    return params, syst_pars

In [ ]:
params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50,
              orbital=True, mu=14, mu_lead=14, mu_sc=100, c_tunnel=3/4,
              V='lambda *_: 0',
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=0,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)


def get_energies(x, key, val, params, syst_pars):
    import common
    import funcs
    import scipy.sparse.linalg as sla
    params = common.parse_params({**params, **val})

    params[key] = x
    
    if params['intrinsic_sc']:
        params, syst_pars = fix_simple_params(params, syst_pars)
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)

    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    energies, _ = sla.eigsh(ham, sigma=0, k=30)
    return energies


vals = common.named_product(orbital=[True, False],
                            intrinsic_sc=[True, False])

learners = []
for val in vals:
    f = partial(get_energies, key='B_x', val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner1D(f, [0, 4], loss_per_interval)
    learner.cdims = val
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)

# Potential shape

In [ ]:
V = lambda x: common.gaussian(x, params['V_0'], syst_pars['L_barrier'], params['sigma'])

hv.Curve([(x, V(x)) for x in range(syst_pars['L'] + syst_pars['L_barrier'])], vdims=['V'])

In [ ]:
runner = adaptive.Runner(learner, client)

In [ ]:
[(l.npoints, l.loss()) for l in learners]

In [ ]:
%%output size=120

def plot(learner):
    Bs = list(learner.data.keys())
    es = np.array([e for e in learner.data.values()])
    return hv.Overlay([hv.Scatter((Bs, e), kdims=['B_x'], vdims=['E']).opts(style=dict(color='k')) for e in es.T])


def plots(learners, vals):
    return hv.HoloMap({tuple(val.values()): plot(l)
                       for l, val in zip(learners, vals)},
                      kdims=list(vals[0].keys()))

plots(learners, vals).select(E=(-.5, .5)).layout().cols(2)

In [ ]:
runner.task.print_stack(), learner.loss()

# Smallest gap phase diagram


In [ ]:
def smallest_gap(x, keys, val, params=params, syst_pars=syst_pars):
    import common
    import funcs
    import numpy as np
    import scipy.sparse.linalg as sla

    params = common.parse_params({**params, **val})

    for _x, key in zip(x, keys):
        params[key] = _x

    if params['intrinsic_sc']:
        params, syst_pars = fix_simple_params(params, syst_pars)
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)

    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    evals, _ = sla.eigsh(ham, sigma=0, k=30, which='LM')
    return np.abs(evals).min()

In [ ]:
params = dict(alpha=70, B_y=0, B_z=0, Delta=60, g=50,
              mu_lead=14, mu_sc=100, c_tunnel=3/4,
              V='lambda *_: 0',
              V_barrier=common.gaussian, sigma=500, V_0=25, x0=0,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False, L_barrier=0,
                 L=3000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = common.named_product(intrinsic_sc=[True, False],
                            orbital=[True, False])

learners = []
for val in vals:
    f = partial(smallest_gap, keys=['B_x', 'mu'],
                val=val,
                params=copy(params),
                syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, ([0, 4], [7, 20]))
    learners.append(learner)
learner = adaptive_tools.BalancingLearner(learners)
common.load_BalancingLearner_data(learners, 'quasi')

In [ ]:
runner = adaptive.Runner(learner, client)

In [ ]:
runner.task.print_stack()

In [ ]:
common.save_BalancingLearner_data(learners, 'quasi')

In [ ]:
[l.n for l in learners]

In [ ]:
def plots(learners, vals, zlims):
    def plot(l):
        im = l.plot(tri_alpha=0)
        im.data = np.log10(np.clip(im.data, *zlims))
        return im
    return hv.HoloMap({tuple(val.values()): plot(l).redim(x='B_x', y='mu')
                       for l, val in zip(learners, vals)},
                      kdims=list(vals[0].keys()))

In [ ]:
%%output size=100
%%opts Image [colorbar=True] 
plots(learners, vals, zlims=(0.01, 1)).layout().cols(2)

In [ ]:
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
def plot(learner):
    ((Ez_min, Ez_max), (mu_min, mu_max)) = learner.bounds

    Ez_min, Ez_max = [Ez * params['mu_B'] * params['g'] / 2 for Ez in [Ez_min, Ez_max]]

    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 1, 1)
    z = np.rot90(learner.plot().data, 3)
    z[z == 0.0] = 1e-5 # Replace zeros as they cannot be plotted in a log plot
    plt.imshow(z.T[::-1], extent=[Ez_min, Ez_max, mu_min, mu_max], aspect='auto', 
               norm=LogNorm(vmin=0.01, vmax=1), interpolation='nearest')
    ax.set_xlim(Ez_min, Ez_max)
    plt.colorbar().set_label(label=r'$E$ (meV)',size=20)
    ax.tick_params(labelsize=20)
    ax.set_xlabel(r'$E_{\mathrm{Z}}$ (meV)', fontsize=15)
    ax.set_ylabel(r'$\mu$ (meV)', fontsize=20)
    return fig, ax

i = 0
fig, ax = plot(learners[i])
ax.set_title(vals[i])
plt.show()

# Conductance

In [ ]:
params['V_0']

In [ ]:
params['mu_lead']

In [ ]:
syst_pars['L_barrier'] = 1000
params['sigma'] = 200

V = lambda x: (0
#     common.gaussian(x, params['V_0'], syst_pars['L_barrier'], params['sigma']) 
               + params['mu_lead'] * (2 + np.tanh((x - syst_pars['L_barrier']) / params['sigma'])))

# V = lambda x: (params['V_0'] * np.tanh((x - syst_pars['L_barrier']) / params['sigma']))

hv.Curve([(x, V(x)) for x in range(syst_pars['L'] + syst_pars['L_barrier'])], vdims=['V'])

In [ ]:
def conductance(x, val, syst_pars, params):
    import funcs

    assert syst_pars['L_barrier'] != 0, "Need a finite length barrier!"

    for k in ['angle']:
        try:
            syst_pars[k] = val[k]
        except KeyError:
            pass

    params = funcs.parse_params(dict(**params, **val))
    
    params['mu_lead'] = params['mu']
    params['B_x'], params['B_y'], params['B_z'] = (0, 0, 0)
    val['V_bias'], params['B_{}'.format(val['direction'])] = x

    if params['intrinsic_sc']:
        params, syst_pars = fix_simple_params(params, syst_pars)
        syst = funcs.make_simple_3d_wire(**syst_pars)
    else:
        syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val['V_bias'])

# Random crap

In [ ]:
# %%opts Path [aspect='square']
params = dict(c_tunnel=3/4, B_x=0.5, B_y=0, B_z=0, V_barrier=40, g=50, mu_sc=100,
              alpha=70, orbital=False, V=lambda x, y, z: 10 * z / 35, Delta=60,
              **funcs.constants.__dict__)

lead_pars = dict(a=10, angle=0,
                 coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_shell=True, A_correction=True)

lead = funcs.make_lead(**lead_pars).finalized()
params['mu_lead'] = params['mu_sc'] = 15
params['B_y'] = 0
ks = np.linspace(-1, 1)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

p1[:, -25:25]# + p1[:, -2:2]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print(f'The bandgap is {gap} meV')

In [ ]:
params = dict(alpha=70, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=15, mu_lead=15, mu_sc=100, c_tunnel=3/4, V=lambda x,y,z:0, V_barrier=50,
              **funcs.constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params, 0)

# Tuning the gap

In [ ]:
params = dict(c_tunnel=3/4, B_x=0, B_y=0, B_z=0, V_barrier=50, g=50,
              alpha=70, orbital=True, V='lambda x, y, z: 10 * z / 35', mu_sc=100,
              **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, coverage_angle=135, r1=35, r2=70,
                 shape='circle', with_shell=True, A_correction=True)

def lowest_energy(x, syst_pars, params):
    import funcs
    import numpy as np
    lead = funcs.make_lead(**syst_pars).finalized()
    params['mu_lead'], params['Delta'] = x
    # Combine the fixed parameters `params` and the changing
    # parameters `val` to one dict and evaluate the string
    # lambda functions.
    params = funcs.parse_params(params)

    # Create the Hamiltonian `ham` at k=0.
    h0 = lead.cell_hamiltonian(params=params)
    t0 = lead.inter_cell_hopping(params=params)
    ham = h0 + t0 + t0.conj().T

    # Find the energies.
    ev = np.linalg.eigvalsh(ham)
    
    # Return a combined dictionary with the results and input.
    return np.abs(ev).min()

def find_crossings(f, g):
    return np.argwhere(np.diff(np.sign(f - g)) != 0).reshape(-1)

def get_Delta(learner, Delta_ind):
    plot = learner.plot(n=1000).redim(x='mu', y='Delta', z='E_gap')
    line = plot.reduce(['mu'], np.min)
    idx = find_crossings(line.data['E_gap'], Delta_ind)[-1]
    Delta = line.data['Delta'][idx]
    return Delta

In [ ]:
learner = adaptive_tools.Learner2D(partial(lowest_energy, syst_pars=syst_pars, params=params),
                             [(5, 20), (0, 200)])

runner = adaptive.Runner(learner, adaptive.Se, goal=lambda l: l.loss() < 0.001)

In [ ]:
runner.task.print_stack()

In [ ]:
%%opts Image [aspect='square']
plot = learner.plot().redim(x='mu', y='Delta', z='E_gap')
plot

In [ ]:
%%opts Scatter (size=10, color='r')
Delta_ind = 0.4

line = plot.reduce(['mu'], np.min)
line * hv.HLine(0.4) * hv.Scatter((get_Delta(learner, Delta_ind), Delta_ind))

In [ ]:
Deltas = np.arange(*learner.bounds[1])
hm = hv.HoloMap({Delta: plot.sample(Delta=Delta)[:, 0:1] for Delta in Deltas})
hm * hv.HLine(0.4)